In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import plotly.graph_objects as go

from scipy.signal import savgol_filter
from scipy.fft import fft, ifft
from scipy.signal import butter,filtfilt

# %matplotlib widget

In [2]:
def modifyTimestamps(df):

    df = df.dropna()
    df = df.drop(df.filter(regex='Name..').columns, axis=1)
    df = df.rename(columns={"Name": "Time"})

    return df


def LDAP_to_time(t_array):

    timestamp_array = []

    for timestamp in t_array:

        if type(timestamp) == pd._libs.tslibs.timestamps.Timestamp:
            timestamp_array.append(timestamp)

        else:
            value = datetime.datetime(1601, 1, 1) + datetime.timedelta(seconds=timestamp/10000000)
            timestamp_array.append(value)

    return np.asarray(timestamp_array)

#### **Loading Data** and **Modifying Time Column**

In [3]:
powerFailureDf = pd.read_csv('Power Anomalies Simulations Data/AD3_data/01_Power_failure_10sec.csv', sep=';')
powerFailureDf = modifyTimestamps(df=powerFailureDf)

#### **Getting 10 cycles of AC signals** and **Converting to timestamps**

In [4]:
start   =   255
end     =   start + 4000

V1  =   powerFailureDf['U_L1real'].iloc[start:end].to_numpy()
I1  =   powerFailureDf['I_L1real'].iloc[start:end].to_numpy()

T   =   powerFailureDf['Time'].iloc[start:end].to_numpy()
T   =   LDAP_to_time(t_array=T)

In [5]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            # x = time.to_numpy(),
            y = V1,
            line =  dict(shape =  'spline', color = 'lightskyblue'),
            name = 'Noisy V1',
            ))

fig.add_trace(go.Scatter(
            y = I1,
            line =  dict(shape =  'spline', color = 'lightsalmon'),
            name = 'Noisy I1'
            ))

fig.show()

#### **Defining Low Pass Filter** and **Filtering V and I signals**

In [6]:
def butter_lowpass_filter(signal, cutoff, fs, order):

    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, signal)
    
    return y

In [7]:
sampling_freq   =   20000.0         # sampling frequency of the signal (Hz)
cutoff_freq     =   100             # desired cutoff frequency of the filter (Hz)
order           =   2               # sine wave can be approx represented as quadratic

V1_filt = butter_lowpass_filter(signal=V1, cutoff=cutoff_freq, fs=sampling_freq, order=order)
I1_filt = butter_lowpass_filter(signal=I1, cutoff=cutoff_freq, fs=sampling_freq, order=order)

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            # x = time.to_numpy(),
            y = V1,
            line =  dict(shape =  'spline', color = 'lightskyblue'),
            name = 'Noisy U1',
            ))

fig.add_trace(go.Scatter(
            y = V1_filt,
            line =  dict(shape =  'spline', color = 'darkslategrey'),
            name = 'Filtered U1'
            ))

fig.show()

In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = I1,
            line =  dict(shape =  'spline', color = 'lightsalmon'),
            name = 'Noisy I1',
            ))

fig.add_trace(go.Scatter(
            y = I1_filt,
            line =  dict(shape =  'spline', color = 'darkslategrey'),
            name = 'Filtered I1',
            ))

fig.show()

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = V1_filt,
            line =  dict(shape =  'spline', color = 'darkcyan'),
            name = 'Filtered V1',
            ))

fig.add_trace(go.Scatter(
            y = I1_filt,
            line =  dict(shape =  'spline', color = 'darksalmon'),
            name = 'Filtered I1',
            ))

fig.show()

#### **Calculating Phase Angle**

In [11]:
def phaseAngle(V_array, I_array, time_array):
    
    zero_crossings_V_filt = list(np.asarray((-0.0025 <= V_array) & (V_array <= 0.0025)).nonzero())[0]
    zero_crossings_I_filt = list(np.asarray((-0.0025 <= I_array) & (I_array <= 0.0025)).nonzero())[0]

    # Get the time stamps when V and I cross zero
    time_stamps_V = time_array[zero_crossings_V_filt]
    time_stamps_I = time_array[zero_crossings_I_filt]

    # Compute the phase angle between V and I signals
    td = pd.Timedelta(seconds=1)
    time_delta = (time_stamps_V[0] - time_stamps_I[0]) / td

    phase_angle_degrees = 360 * 50 * time_delta

    return phase_angle_degrees

In [12]:
phase_angle_degrees = phaseAngle(V_array=V1_filt, I_array=I1_filt, time_array=T)

print(f'Phase Angle: {phase_angle_degrees} degrees')

Phase Angle: -3.6 degrees


#### **Calculating Power**

In [13]:
def power(V_array, I_array, phase_angle):

    power_array = np.multiply(V_array, I_array) * np.cos(phase_angle)
    return power_array

In [14]:
P1 = power(V_array=V1_filt, I_array=I1_filt, phase_angle=phase_angle_degrees)
P1

array([-0.00027931, -0.00033881, -0.00040369, ..., -0.00989019,
       -0.00987086, -0.00985466])

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(
            y = P1,
            line =  dict(shape =  'spline', color = 'darkcyan'),
            name = 'P1',
            ))

fig.show()

#### **Calculating FFT** and **Harmonics present in the signal**

In [49]:
def FFT(signal, fs=20000):

    X = fft(signal)
    N = len(X)
    n = np.arange(N)
    T = N/fs

    freq = n/T 
    freq_amplitudes = np.abs(X)

    sorted_indices = np.argsort(-freq_amplitudes[:int(N/2)])

    dominant_freq = freq[sorted_indices[0]]
    other_freq = freq[sorted_indices[1:]]

    return (freq, freq_amplitudes, dominant_freq, other_freq)

In [50]:
freq, freq_amplitudes, dominant_freq, other_freq = FFT(signal=V1_filt, fs=sampling_freq)

print(f'Prime Harmonic: {dominant_freq} Hz')
print(f'Next 5 Harmonics: {other_freq[:5]}')

fig = go.Figure()

fig.add_trace(go.Scatter(x=freq, y=freq_amplitudes, mode='lines', line=dict(color='blue'), name='FFT Amplitude |X(freq)|'))
fig.update_layout(
    title='FFT Visualization',
    xaxis_title='Freq (Hz)',
    yaxis_title='FFT Amplitude |X(freq)|',
    xaxis=dict(range=[0, sampling_freq/2])
)

fig.show()

Prime Harmonic: 50.0 Hz
Next 5 Harmonics: [  0. 150.  75.  65.  95.]
